# Trends

This notebook looks at rates and trends in post-fire harvests between years, ecoregions, 
ownerships, and severity levels.

In [ ]:
import pandas as pd

from pfh.scripts.config import OWNER_GROUPS

## Data Prep

Load the stratified harvest area results exported from Earth Engine and process into annual trends and summaries.

In [ ]:
df = pd.read_csv("../data/results/stratified_results.csv").drop(
    columns=["system:index", ".geo"]
)

# Remove unmanaged lands from analysis
df = df[~df["owner"].isin(["wilderness", "nps"])]

# Assign owner groups
df["owner_group"] = df["owner"].map(OWNER_GROUPS)

# Assign severity categories
df["severity"] = pd.Categorical(
    df["severity"], categories=["Very low", "Low", "Moderate", "High"], ordered=True
)
# Check for incorrect severity category names which will produce NAs
assert not df.isna().any().any(), "Check severity category names!"

# Collapse all timings to simplify future processing where having multiple timings
# could lead to multi-counting areas.
df_all_timings = (
    df.groupby(["event_id", "owner", "severity"], observed=False)
    .agg({
        "harvest_area": "sum",
        "analysis_area": "first",
        "year": "first",
        "ecoregion": "first",
    })
    .reset_index()
)

### Annual Trends

Calculate annual trends by owner and severity class from the raw data.

In [ ]:
# Annual harvest trends by individual owner (e.g. USFS) and severity class
owner_trends = (
    df_all_timings.drop(columns=["event_id", "ecoregion"])
    .groupby(["year", "owner", "severity"], observed=False)
    .agg("sum")
    .reset_index()
)

# Annual harvest trends by owner group (e.g. federal) and severity class
owner_group_trends = (
    owner_trends.assign(owner=lambda x: x["owner"].map(OWNER_GROUPS))
    .groupby(["year", "owner", "severity"], observed=False)
    .agg("sum")
    .reset_index()
)

# Calculate trends for all owners
all_owner_trends = (
    owner_trends.groupby(["year", "severity"], observed=False)
    .agg("sum")
    .reset_index()
    .assign(owner="Total")
)

# Append the "All" owner group to the owner group trends
owner_group_trends = pd.concat([owner_group_trends, all_owner_trends])
owner_group_trends["harvest_rate"] = (
    owner_group_trends["harvest_area"] / owner_group_trends["analysis_area"]
).fillna(0)

In [ ]:
owner_group_trends["cumulative_harvest_area"] = owner_group_trends.groupby(
    ["owner", "severity"], observed=False
).harvest_area.cumsum()

owner_group_trends

Export annual trends by owner and severity class for time series analysis in R.

In [ ]:
owner_trends.to_csv("../data/results/owner_trends.csv", index=False)
owner_group_trends.to_csv("../data/results/owner_group_trends.csv", index=False)

## Results

### Harvest Timing

Summarize timing of harvests overall and by ownerships.

In [ ]:
timing_df = (
    df.drop(columns=["ecoregion", "year"])
    .groupby(["timing", "owner_group", "severity"], observed=False)
    .agg("sum")
    .reset_index()
)

owner_timing_summary = (
    timing_df.groupby(["timing", "owner_group"])
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
)
total_salvage = (
    owner_timing_summary.groupby(["timing"])
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
    .assign(owner_group="Total")
)
owner_timing_summary = pd.concat(
    [owner_timing_summary, total_salvage], axis=0, ignore_index=True
).sort_values(["owner_group", "timing"])

owner_timing_summary["percent_of_harvest"] = (
    owner_timing_summary.groupby("owner_group")["harvest_area"]
    .apply(lambda x: x / x.sum())
    .values
)

Print summary stats by owner.

In [ ]:
for owner in ["Total", "Federal", "Private"]:
    owner_timing = owner_timing_summary[owner_timing_summary.owner_group.eq(owner)]
    pct_in_y1 = owner_timing[owner_timing.timing.eq(1)].percent_of_harvest.values[0]
    pct_after_y3 = 1 - owner_timing[owner_timing.timing.lt(4)].percent_of_harvest.sum()

    print(f"{owner}: {pct_in_y1:.0%} in year 1, {pct_after_y3:.0%} after year 3")

### Total Area

How much total harvest area was predicted?

In [ ]:
harvest_ha = owner_trends.harvest_area.sum()
analysis_ha = owner_trends.analysis_area.sum()
overal_harvest_rate = harvest_ha / analysis_ha

print(
    f"We mapped {harvest_ha / 1e3:.0f}k hectares of post-fire harvest across "
    f"{analysis_ha / 1e6:.1f}M hectares of burned forest ({overal_harvest_rate:.1%})."
)

### Area Summary by Owner

Summarize total area analyzed and harvests by owner and owner groups.

In [ ]:
owner_summary = (
    owner_trends.groupby(["owner"])
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
    .assign(pct_of_total=lambda x: x["analysis_area"] / x["analysis_area"].sum())
    .assign(harvest_rate=lambda x: x["harvest_area"] / x["analysis_area"])
)

owner_group_summary = (
    owner_group_trends[owner_group_trends.owner.ne("Total")]
    .copy()
    .groupby(["owner"])
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
    .assign(pct_of_total=lambda x: x["analysis_area"] / x["analysis_area"].sum())
    .assign(harvest_rate=lambda x: x["harvest_area"] / x["analysis_area"])
)

In [ ]:
owner_summary

In [ ]:
owner_group_summary

In [ ]:
owner_group_summary.harvest_area.sum() / owner_group_summary.analysis_area.sum()

### Area Trends by Ecoregion

Calculate annual trends by ecoregion.

In [ ]:
ecoregion_trends = (
    df_all_timings.assign(owner_group=df_all_timings.owner.map(OWNER_GROUPS))
    .groupby(["ecoregion", "owner_group", "year"])
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
)

ecoregion_trends.to_csv("../data/results/ecoregion_owner_trends.csv", index=False)

Summarize trends by ecoregion.

In [ ]:
ecoregion_summary = (
    ecoregion_trends.groupby("ecoregion")
    .agg({"harvest_area": "sum", "analysis_area": "sum"})
    .reset_index()
    .assign(harvest_rate=lambda x: x["harvest_area"] / x["analysis_area"])
    .assign(pct_of_analysis=lambda x: x["analysis_area"] / x["analysis_area"].sum())
)

ecoregion_summary